# <font color="red">**3. RF FS - Fine-tuning Hyperparameters**</font>

**Author:** Osmar Bolivar

In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go

## **1. Monthly data**

In [140]:
raw = pd.read_csv("./Data/RF_DATASET.csv", index_col=0)
raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5901 entries, 2010-01-01 to 2024-10-31
Data columns (total 54 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ipc_all       178 non-null    float64
 1   lag_1         5901 non-null   float64
 2   lag_2         5901 non-null   float64
 3   lag_3         5901 non-null   float64
 4   lag_6         5901 non-null   float64
 5   lag_9         5901 non-null   float64
 6   lag_12        5901 non-null   float64
 7   oil_cb        5901 non-null   float64
 8   sugar_bol     5901 non-null   float64
 9   chicken_tj    5901 non-null   float64
 10  oil2_tj       5901 non-null   float64
 11  oil2_bol      5901 non-null   float64
 12  sugar_tj      5901 non-null   float64
 13  rice4_tj      5901 non-null   float64
 14  sugar_cb      5901 non-null   float64
 15  rice3_po      5901 non-null   float64
 16  sugar_or      5901 non-null   float64
 17  sugar_sc      5901 non-null   float64
 18  rice4_su      5901

In [141]:
df = raw.copy().query('freq == "month"')
df.drop(columns=["freq"], inplace=True)
df

,ipc_all,lag_1,lag_2,lag_3,lag_6,lag_9,lag_12,oil_cb,sugar_bol,chicken_tj,...,chicken_sc,tomato_co,libor,platano_co,sugar_po,carrot_co,tomato_tj,oil_tr,lard_lp,rice4_or
2010-01-31,0.074472,0.263790,0.457498,0.784931,1.447299,5.323961,11.041182,8.829032,173.290323,12.822581,...,9.494194,174.0,0.400828,75.000000,174.032258,62.500000,44.303871,9.170000,185.0,250.00
2010-02-28,0.312381,0.074472,0.263790,0.457498,1.417620,3.200128,8.128586,8.800000,183.071429,12.714286,...,11.184286,174.0,0.387948,75.000000,184.107143,62.500000,53.852143,9.170000,185.0,250.00
2010-03-31,0.685510,0.312381,0.074472,0.263790,0.643999,2.117642,6.563669,8.800000,180.000000,13.112903,...,11.458065,174.0,0.411916,75.000000,183.322581,62.500000,46.201290,9.170000,185.0,250.00
2010-04-30,1.215574,0.685510,0.312381,0.074472,0.784931,1.447299,5.323961,8.800000,184.833333,13.500000,...,12.020000,174.0,0.470366,75.666667,188.300000,62.500000,44.731000,9.169333,185.0,250.00
2010-05-31,1.384706,1.215574,0.685510,0.312381,0.457498,1.417620,3.200128,8.800000,196.451613,13.032258,...,10.316129,174.0,0.655943,71.612903,196.290323,58.870968,38.882258,8.931290,185.0,250.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-30,3.844356,3.516994,3.460018,3.063085,2.118325,2.847826,2.725282,10.500000,245.000000,13.500000,...,13.641667,110.0,5.709759,66.670000,240.000000,55.000000,94.752000,10.800000,232.5,10.17
2024-07-31,3.953707,3.844356,3.516994,3.460018,1.863150,2.075432,2.700872,10.478710,242.161290,13.935484,...,13.751613,110.0,5.590282,66.670000,240.000000,55.000000,81.698387,10.800000,232.5,10.17
2024-08-31,5.192869,3.953707,3.844356,3.516994,2.516604,1.594145,3.058496,11.132258,241.161290,14.000000,...,13.932258,110.0,5.238481,66.670000,240.000000,55.000000,94.300000,10.800000,232.5,10.17
2024-09-30,6.182067,5.192869,3.953707,3.844356,3.063085,2.118325,2.847826,11.660000,242.833333,14.000000,...,13.316667,110.0,4.939704,66.670000,240.000000,55.000000,94.300000,10.800000,232.5,10.17


In [142]:
isna = df.isna().sum()
isna[isna > 0]

Series([], dtype: int64)

In [143]:
df.shape

(178, 53)

## **2. Train and Val sets**

In [144]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [145]:
train_size = int(len(df) * 0.8)
train, validation = df[:train_size], df[train_size:]
print(f'Obs in train set: {train.shape[0]}; variables in train set: {train.shape[1]}')
print(f'Obs in validation set: {validation.shape[0]}; variables in validation set: {validation.shape[1]}')

Obs in train set: 142; variables in train set: 53
Obs in validation set: 36; variables in validation set: 53


In [146]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 2021-11-30 to 2024-10-31
Data columns (total 53 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ipc_all       36 non-null     float64
 1   lag_1         36 non-null     float64
 2   lag_2         36 non-null     float64
 3   lag_3         36 non-null     float64
 4   lag_6         36 non-null     float64
 5   lag_9         36 non-null     float64
 6   lag_12        36 non-null     float64
 7   oil_cb        36 non-null     float64
 8   sugar_bol     36 non-null     float64
 9   chicken_tj    36 non-null     float64
 10  oil2_tj       36 non-null     float64
 11  oil2_bol      36 non-null     float64
 12  sugar_tj      36 non-null     float64
 13  rice4_tj      36 non-null     float64
 14  sugar_cb      36 non-null     float64
 15  rice3_po      36 non-null     float64
 16  sugar_or      36 non-null     float64
 17  sugar_sc      36 non-null     float64
 18  rice4_su      36 non

In [147]:
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)
validation_scaled = pd.DataFrame(scaler.transform(validation), columns=validation.columns, index=validation.index)
#test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

X_train = train_scaled.drop('ipc_all', axis=1)
#y_train = train_scaled['ipc_all']
y_train = train['ipc_all']

X_validation = validation_scaled.drop('ipc_all', axis=1)
#y_validation = validation_scaled['ipc_all']
y_validation = validation['ipc_all']

#X_test = test_scaled.drop('ipc_all', axis=1)

## **3. Algorithms**

In [148]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

tscv5 = TimeSeriesSplit(n_splits=5)

### **3.1. Ridge**   

**Without tuning (Validation):**
MSE:  0.178; R2:  0.436; MAE:  0.273

In [149]:
#ridge = Ridge(random_state=0)
ridge = Ridge(alpha=3.5, fit_intercept=True, positive=False, random_state=0) 
# Fit on training set
ridge.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
ridge_train_pred = ridge.predict(X_train)
ridge_val_pred = ridge.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ridge = mean_squared_error(y_train, ridge_train_pred)
r2_train_ridge = r2_score(y_train, ridge_train_pred)
mae_train_ridge = mean_absolute_error(y_train, ridge_train_pred)
print("Train MSE: ", mse_train_ridge)
print("Train R2: ", r2_train_ridge)
print("Train MAE: ", mae_train_ridge)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

Train MSE:  0.12930133316763595
Train R2:  0.981529417300062
Train MAE:  0.27746099782303923
Validation MSE:  0.4405197048613215
Validation R2:  0.8006760187383335
Validation MAE:  0.5075875864976575


In [ ]:
# Define parameter grid for GridSearchCV
alphas_ridge = np.logspace(0.1, 10, num=500)
#alphas_ridge = np.arange(0.5, 7, 0.5)
param_grid = {
    'alpha': alphas_ridge,  # [1.0] 
    'fit_intercept': [True],  # [True, False] 
    'copy_X': [True], 
    'max_iter': [10000], 
    'tol': [0.0001], 
    'solver': ['auto'], 
    'positive': [False],  # [True, False]
    'random_state': [0]    
}

# Instantiate Ridge model
ridge = Ridge()
# Define GridSearchCV object
grid_search_ridge = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error') 
# Fit GridSearchCV on training set
grid_search_ridge.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_ridge.best_params_)
print("Best score: ", -grid_search_ridge.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_ridge = grid_search_ridge.best_estimator_
ridge_val_pred = best_ridge.predict(X_validation)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

In [150]:
# Get the coefficients from the Ridge model
coef = ridge.coef_
# Create a dataframe of feature importances
feature_importance_ridge = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_ridge = feature_importance_ridge.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_ridge.columns = ['feat_ridge', 'imp_ridge']
# Print the feature importances
feature_importance_ridge.head(15)

,feat_ridge,imp_ridge
0,lag_1,1.021572
1,rice3_su,0.349202
2,squash_co,0.339682
3,flour_lp,0.276005
4,oil_cb,0.273724
5,lag_6,0.256779
6,silver,0.216968
7,chicken_tr,0.191472
8,platano_co,0.180314
9,oil2_bol,0.160169


### **3.2. Lasso**
Without tuning:
Validation MSE:  0.796
Validation R2:  0.640
Validation MAE:  0.689

In [151]:
#lasso = Lasso(random_state=0)
lasso = Lasso(alpha=0.37, fit_intercept=True, max_iter=10000, random_state=0)  
# Fit on training set
lasso.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
lasso_train_pred = lasso.predict(X_train)
lasso_val_pred = lasso.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_lasso = mean_squared_error(y_train, lasso_train_pred)
r2_train_lasso = r2_score(y_train, lasso_train_pred)
mae_train_lasso = mean_absolute_error(y_train, lasso_train_pred)
print("Train MSE: ", mse_train_lasso)
print("Train R2: ", r2_train_lasso)
print("Train MAE: ", mae_train_lasso)
# Calculate Forecast metrics on validation set
mse_val_lasso = mean_squared_error(y_validation, lasso_val_pred)
r2_val_lasso = r2_score(y_validation, lasso_val_pred)
mae_val_lasso = mean_absolute_error(y_validation, lasso_val_pred)
print("Validation MSE: ", mse_val_lasso)
print("Validation R2: ", r2_val_lasso)
print("Validation MAE: ", mae_val_lasso)

Train MSE:  0.503905461775445
Train R2:  0.9280175441609182
Train MAE:  0.5402481153738985
Validation MSE:  0.3920554625423185
Validation R2:  0.8226048578373584
Validation MAE:  0.45544558626053777


In [ ]:
# Define parameter grid for GridSearchCV
alphas_lasso = np.logspace(-1, 2, num=500)
#alphas_lasso = np.arange(0.1, 100, 0.05)
param_grid = {
    'alpha': [0.4],  # alphas_lasso [1.0]
    'fit_intercept': [True], 
    'precompute': [False], 
    'copy_X': [True], 
    'max_iter': [10000],  # [1000]
    'tol': [0.0001], 
    'warm_start': [False], 
    'positive': [False], 
    'random_state': [0], 
    'selection': ['cyclic']
}

# Instantiate lasso model
lasso = Lasso()
# Define GridSearchCV object
grid_search_lasso = GridSearchCV(lasso, param_grid, cv=tscv5, scoring='neg_mean_squared_error')  
# Fit GridSearchCV on training set
grid_search_lasso.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_lasso.best_params_)
print("Best score: ", -grid_search_lasso.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_lasso = grid_search_lasso.best_estimator_
y_val_pred = best_lasso.predict(X_validation)
# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Make predictions on test set using best model from GridSearchCV
#y_test_pred_lasso = best_lasso.predict(X_test)

Best parameter:  {'alpha': 0.4, 'copy_X': True, 'fit_intercept': True, 'max_iter': 10000, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
Best score:  0.666270826211424
Validation MSE:  0.39412530274519775
Validation R2:  0.8216683077006437
Validation MAE:  0.45071746545346986


In [152]:
# Get the coefficients from the Lasso model
coef = lasso.coef_
# Create a dataframe of feature importances
feature_importance_lasso = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_lasso = feature_importance_lasso.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_lasso.columns = ['feat_lasso', 'imp_lasso']
# Print the feature importances
feature_importance_lasso.head(15)

,feat_lasso,imp_lasso
0,lag_1,2.094728
1,sugar_lp,0.114949
2,oil_tr,0.028830
3,chicken_po,0.000000
4,pineapple_tr,-0.000000
5,rice2_lp,0.000000
6,rice3_su,0.000000
7,ufv,-0.000000
8,paprika_or,-0.000000
9,squash_co,0.000000


### **3.3. ElasticNet**
Without tuning:
Validation MSE:  1.456
Validation R2:  0.350
Validation MAE:  1.120

In [153]:
enet = ElasticNet(random_state=0)
#enet = ElasticNet(alpha=0.4, l1_ratio=0.97, fit_intercept=True, max_iter=10000, random_state=0) 
# Fit on training set
enet.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
enet_train_pred = enet.predict(X_train)
enet_val_pred = enet.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_enet = mean_squared_error(y_train, enet_train_pred)
r2_train_enet = r2_score(y_train, enet_train_pred)
mae_train_enet = mean_absolute_error(y_train, enet_train_pred)
print("Train MSE: ", mse_train_enet)
print("Train R2: ", r2_train_enet)
print("Train MAE: ", mae_train_enet)
# Calculate Forecast metrics on validation set
mse_val_enet = mean_squared_error(y_validation, enet_val_pred)
r2_val_enet = r2_score(y_validation, enet_val_pred)
mae_val_enet = mean_absolute_error(y_validation, enet_val_pred)
print("Validation MSE: ", mse_val_enet)
print("Validation R2: ", r2_val_enet)
print("Validation MAE: ", mae_val_enet)

Train MSE:  1.0707428120999791
Train R2:  0.8470453229154564
Train MAE:  0.8052740635083214
Validation MSE:  1.4359463705018145
Validation R2:  0.3502707273065909
Validation MAE:  1.1199020599694829


In [ ]:
# Define parameter grid for GridSearchCV
#alphas_enet = np.logspace(-1, 2, num=500)
alphas_enet = np.arange(0.1, 10, 0.05)
param_grid = {
    'alpha': [1.0], # alphas_enet
    'l1_ratio': [0.5], # np.arange(0.1, 0.95, 0.05)
    'fit_intercept': [True], 
    'precompute': [False], 
    'max_iter': [10000], 
    'copy_X': [True], 
    'tol': [0.0001], 
    'warm_start': [False], 
    'positive': [False], 
    'random_state': [0], 
    'selection': ['cyclic']
}

# Instantiate enet model
enet = ElasticNet()
# Define GridSearchCV object
grid_search_enet = GridSearchCV(enet, param_grid, cv=tscv5, scoring='neg_mean_squared_error') 
# Fit GridSearchCV on training set
grid_search_enet.fit(X_train, y_train)
# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_enet.best_params_)
print("Best score: ", -grid_search_enet.best_score_)
# Make predictions on validation set using best model from GridSearchCV
best_enet = grid_search_enet.best_estimator_
y_val_pred = best_enet.predict(X_validation)
# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Make predictions on test set using best model from GridSearchCV
#y_test_pred_enet = best_enet.predict(X_test)

Best parameter:  {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.5, 'max_iter': 10000, 'positive': False, 'precompute': False, 'random_state': 0, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
Best score:  1.7311707199004296
Validation MSE:  1.4359463705018145
Validation R2:  0.3502707273065909
Validation MAE:  1.1199020599694829


In [154]:
# Get the coefficients from the enet model
coef = enet.coef_
# Create a dataframe of feature importances
feature_importance_enet = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_enet = feature_importance_enet.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_enet.columns = ['feat_enet', 'imp_enet']
# Print the feature importances
feature_importance_enet.head(15)

,feat_enet,imp_enet
0,lag_1,0.594116
1,lag_2,0.357928
2,lag_3,0.168861
3,chicken_tj,0.107705
4,sugar_lp,0.097207
5,oil_tr,0.090412
6,sugar_bol,0.084321
7,sugar_tj,0.080156
8,oil2_bol,0.078466
9,sugar_cb,0.077288


### **3.4. ADA**
Without tuning:
Validation MSE:  0.106
Validation R2:  0.663
Validation MAE:  0.247

In [155]:
#ada = AdaBoostRegressor(random_state=0)
ada = AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=5), loss='square', learning_rate=0.98, random_state=0)
# Fit the model on training data
ada.fit(X_train, y_train)
# Make predictions on the validation set
ada_train_pred = ada.predict(X_train)
ada_val_pred = ada.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ada = mean_squared_error(y_train, ada_train_pred)
r2_train_ada = r2_score(y_train, ada_train_pred)
mae_train_ada = mean_absolute_error(y_train, ada_train_pred)
print("Train MSE: ", mse_train_ada)
print("Train R2: ", r2_train_ada)
print("Train MAE: ", mae_train_ada)
# Evaluate the model on the validation set
mse_val_ada = mean_squared_error(y_validation, ada_val_pred)
r2_val_ada = r2_score(y_validation, ada_val_pred)
mae_val_ada = mean_absolute_error(y_validation, ada_val_pred)
print("Validation MSE: ", mse_val_ada)
print("Validation R2: ", r2_val_ada)
print("Validation MAE: ", mae_val_ada)

Train MSE:  0.047793483955778496
Train R2:  0.9931727424899875
Train MAE:  0.15139403322902562
Validation MSE:  0.28355654176437656
Validation R2:  0.871697864605039
Validation MAE:  0.44543392942681


In [ ]:
# Define the AdaBoost Regressor
ada = AdaBoostRegressor()
# Define the range of hyperparameters to search over
param_grid_ada = {
    'estimator': [DecisionTreeRegressor(max_depth=3),  # [None]
                  DecisionTreeRegressor(max_depth=4),
                  DecisionTreeRegressor(max_depth=5),
                  DecisionTreeRegressor(max_depth=6),
                  DecisionTreeRegressor(max_depth=7),
                  DecisionTreeRegressor(max_depth=8) ],   
    'n_estimators': [50],  # range(50, 200, 5)
    'learning_rate': [1.0], # np.logspace(-5,1, 100)
    'loss': ['square'], # ['linear']
    'random_state': [0]
}

# Tune hyperparameters using GridSearchCV with TimeSeriesSplit
grid_search_ada = GridSearchCV(estimator=ada, param_grid=param_grid_ada, cv=tscv5, scoring='neg_mean_squared_error')
grid_search_ada.fit(X_train, y_train)
# Evaluate the model using the best hyperparameters on the test set
ada_best = AdaBoostRegressor(**grid_search_ada.best_params_)
ada_best.fit(X_train, y_train)
# Make predictions on the validation set
y_val_pred = ada_best.predict(X_validation)
# Evaluate the model on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Print the best hyperparameters and the best score
print("Best parameters found: ", grid_search_ada.best_params_)
print("Lowest MSE found: ", -grid_search_ada.best_score_)
# Evaluate the model on the test set
#y_test_pred_ada = ada_best.predict(X_test)

Validation MSE:  0.5715897483639102
Validation R2:  0.7413701520386811
Validation MAE:  0.6529235294133032
Best parameters found:  {'estimator': DecisionTreeRegressor(max_depth=8), 'learning_rate': 1.0, 'loss': 'square', 'n_estimators': 50, 'random_state': 0}
Lowest MSE found:  1.1337376287203764


In [156]:
# Create a DataFrame with the feature importance values
feature_importance_ada = pd.DataFrame({'Feature': X_train.columns, 'Importance': ada.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_ada = feature_importance_ada.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_ada.columns = ['feat', 'imp_ada']
# Print the feature importance DataFrame
feature_importance_ada.head(15)

,feat,imp_ada
0,lag_1,0.606261
1,oil2_bol,0.079094
2,chicken_tj,0.058634
3,chicken_bol,0.026050
4,lag_2,0.020053
5,oil_tr,0.018948
6,sugar_lp,0.017309
7,flour_lp,0.016267
8,sugar_tj,0.015801
9,papaya_tr,0.014125


### **3.5. GBR**
Without tuning:
Validation MSE:  0.557
Validation R2:  0.748
Validation MAE:  0.662

In [157]:
#gbr = GradientBoostingRegressor(random_state=0)
gbr = GradientBoostingRegressor(learning_rate=0.5, n_estimators=110, random_state=0)
# Fit the model on the training set
gbr.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
gbr_train_pred = gbr.predict(X_train)
gbr_val_pred = gbr.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_gbr = mean_squared_error(y_train, gbr_train_pred)
r2_train_gbr = r2_score(y_train, gbr_train_pred)
mae_train_gbr = mean_absolute_error(y_train, gbr_train_pred)
print("Train MSE: ", mse_train_gbr)
print("Train R2: ", r2_train_gbr)
print("Train MAE: ", mae_train_gbr)
# Calculate Forecast metrics on validation set
mse_val_gbr = mean_squared_error(y_validation, gbr_val_pred)
r2_val_gbr = r2_score(y_validation, gbr_val_pred)
mae_val_gbr = mean_absolute_error(y_validation, gbr_val_pred)
print("Validation MSE: ", mse_val_gbr)
print("Validation R2: ", r2_val_gbr)
print("Validation MAE: ", mae_val_gbr)

Train MSE:  1.898096870914645e-07
Train R2:  0.9999999728858517
Train MAE:  0.00033971017159209395
Validation MSE:  0.3068087506640748
Validation R2:  0.8611768304722416
Validation MAE:  0.46375356692150554


In [ ]:
# Define the model
gbr = GradientBoostingRegressor()
# Define the hyperparameters to be tuned
params = {
    'loss': ['squared_error'], # ['squared_error', 'absolute_error', 'huber', 'quantile']
    'learning_rate': [0.5], # [0.1] 
    'n_estimators': [110],  # range(100, 300, 10)
    'subsample': [1.0], 
    'criterion': ['friedman_mse'], # ['friedman_mse', 'squared_error']
    'min_samples_split': [2], 
    'min_samples_leaf': [1], 
    'min_weight_fraction_leaf': [0.0], 
    'max_depth': [3], # range(3,6,1)
    'min_impurity_decrease': [0.0], 
    'init': [None], 
    'random_state': [0], 
    'max_features': [None], 
    'alpha': [0.9], 
    'verbose': [0], 
    'max_leaf_nodes': [None], 
    'warm_start': [False], 
    'validation_fraction': [0.1], 
    'n_iter_no_change': [None], 
    'tol': [0.0001], 
    'ccp_alpha': [0.0]
}

# Create the GridSearchCV object
grid_gbr = GridSearchCV(gbr, params, cv=tscv5, scoring='neg_mean_squared_error', n_jobs=-1)
# Fit the model on the training set with GridSearchCV
grid_gbr.fit(X_train, y_train)
# Print the best hyperparameters
print('Best hyperparameters:', grid_gbr.best_params_)
# Use the best model to make predictions on the validation set
y_val_pred = grid_gbr.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)
# Use the best model to make predictions on the test set
#y_test_pred_gbr = grid_gbr.predict(X_test)

Best hyperparameters: {'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.5, 'loss': 'squared_error', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 110, 'n_iter_no_change': None, 'random_state': 0, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
Validation MSE:  0.3068087506640748
Validation R2:  0.8611768304722416
Validation MAE:  0.46375356692150554


In [158]:
# Create a DataFrame with the feature importance values
feature_importance_gbr = pd.DataFrame({'Feature': X_train.columns, 'Importance': gbr.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_gbr = feature_importance_gbr.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_gbr.columns = ['feat_gbr', 'imp_gbr']
# Print the feature importance DataFrame
feature_importance_gbr.head(15)

,feat_gbr,imp_gbr
0,lag_1,0.775894
1,chicken_tj,0.101247
2,oil2_bol,0.026041
3,sugar_tj,0.025934
4,libor,0.010133
5,ycorn_sc,0.008404
6,paprika_or,0.008075
7,ufv,0.006513
8,tomato_tj,0.006256
9,papa1_po,0.005580


### **3.4. RF**
Without tuning:
Validation MSE:  0.446
Validation R2:  0.798
Validation MAE:  0.576

In [159]:
#rf = RandomForestRegressor(random_state=0)
rf = RandomForestRegressor(max_depth=6, criterion="absolute_error", random_state=0)

# Fit the model to the training data and make predictions on the validation set
rf.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
rf_train_pred = rf.predict(X_train)
rf_val_pred = rf.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_rf = mean_squared_error(y_train, rf_train_pred)
r2_train_rf = r2_score(y_train, rf_train_pred)
mae_train_rf = mean_absolute_error(y_train, rf_train_pred)
print("Train MSE: ", mse_train_rf)
print("Train R2: ", r2_train_rf)
print("Train MAE: ", mae_train_rf)
# Calculate Forecast metrics on validation set
mse_val_rf = mean_squared_error(y_validation, rf_val_pred)
r2_val_rf = r2_score(y_validation, rf_val_pred)
mae_val_rf = mean_absolute_error(y_validation, rf_val_pred)
print("Validation MSE: ", mse_val_rf)
print("Validation R2: ", r2_val_rf)
print("Validation MAE: ", mae_val_rf)

Train MSE:  0.05402277141480336
Train R2:  0.9922828942080334
Train MAE:  0.1796417670973958
Validation MSE:  0.28193770056143586
Validation R2:  0.8724303491455481
Validation MAE:  0.4489060362638738


In [ ]:
# Define the Random Forest Regression model
rf_reg = RandomForestRegressor()

# Define the hyperparameters to tune
param_grid_rf = {
    'n_estimators': [100],  # range(100, 300, 10)
    'criterion': ['absolute_error'], 
    'max_depth': [None], 
    'min_samples_split': [2],  # range(2,20,1) 
    'min_samples_leaf': [1], 
    'min_weight_fraction_leaf': [0.0], 
    'max_features': [1.0], 
    'max_leaf_nodes': [None], 
    'min_impurity_decrease': [0.0], 
    'bootstrap': [True], 
    'oob_score': [False], 
    'n_jobs':[None], 
    'random_state': [0], 
    'verbose': [0], 
    'warm_start': [False], 
    'ccp_alpha': [0.0], 
    'max_samples': [None], 
    'monotonic_cst': [None]
}

# Define the GridSearchCV object
grid_rf_reg = GridSearchCV(estimator=rf_reg, param_grid=param_grid_rf, cv=tscv5, scoring='neg_mean_squared_error')
# Fit the GridSearchCV object to the training data
grid_rf_reg.fit(X_train, y_train)
# Extract the best hyperparameters and score
best_params = grid_rf_reg.best_params_
best_score = grid_rf_reg.best_score_
# Print the best hyperparameters found by GridSearchCV
print(f"Best hyperparameters: {best_params}")
print(f"Best score: {best_score}")
# Instantiate a new Random Forest Regression model using the best hyperparameters
rf_reg_best = RandomForestRegressor(**best_params)
# Fit the model to the training data and make predictions on the validation set
rf_reg_best.fit(X_train, y_train)
# Use the best model to make predictions on the validation set
y_val_pred = rf_reg_best.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_rf = rf_reg_best.predict(X_test)

Best hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'absolute_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}
Best score: -1.3678690599324081
Validation MSE:  0.30845034723866915
Validation R2:  0.8604340497038383
Validation MAE:  0.47549773637424686


In [160]:
# Create a DataFrame with the feature importance values
feature_importance_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_rf = feature_importance_rf.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_rf.columns = ['feat', 'imp_rf']
# Print the feature importance DataFrame
feature_importance_rf.head(30)

,feat,imp_rf
0,lag_1,0.563287
1,oil2_bol,0.055866
2,chicken_tj,0.054483
3,flour_lp,0.029166
4,sugar_tj,0.021121
5,papa1_po,0.017184
6,sugar_lp,0.016289
7,chicken_bol,0.013197
8,libor,0.013182
9,rice_bol,0.012815


### **3.5. ET**   
Without tuning:
Validation MSE:  0.995
Validation R2:  0.550
Validation MAE:  0.838

In [161]:
#et = ExtraTreesRegressor(random_state=0)
et = ExtraTreesRegressor(max_depth=9, criterion='absolute_error', bootstrap=True, oob_score=True, random_state=0)
# Fit the model to the training data and make predictions on the validation set
et.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
et_train_pred = et.predict(X_train)
et_val_pred = et.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_et = mean_squared_error(y_train, et_train_pred)
r2_train_et = r2_score(y_train, et_train_pred)
mae_train_et = mean_absolute_error(y_train, et_train_pred)
print("Train MSE: ", mse_train_et)
print("Train R2: ", r2_train_et)
print("Train MAE: ", mae_train_et)
# Calculate Forecast metrics on validation set
mse_val_et = mean_squared_error(y_validation, et_val_pred)
r2_val_et = r2_score(y_validation, et_val_pred)
mae_val_et = mean_absolute_error(y_validation, et_val_pred)
print("Validation MSE: ", mse_val_et)
print("Validation R2: ", r2_val_et)
print("Validation MAE: ", mae_val_et)

Train MSE:  0.03912007327039241
Train R2:  0.9944117316436976
Train MAE:  0.15098283073145718
Validation MSE:  0.5515549455008588
Validation R2:  0.7504353916327037
Validation MAE:  0.6020106948330262


In [ ]:
# Define the Extra Trees Regression model
et_reg = ExtraTreesRegressor()
# Define the hyperparameter grid to search over
param_grid = {
    'n_estimators': [100], # range(100, 300, 15)
     'criterion': ['squared_error'], 
     'max_depth':[None], 
     'min_samples_split':[2], 
     'min_samples_leaf':[1], 
     'min_weight_fraction_leaf':[0.0], 
     'max_features':[1.0], 
     'max_leaf_nodes':[None], 
     'min_impurity_decrease':[0.0], 
     'bootstrap':[True], # [False] 
     'oob_score':[True], # [False]
     'n_jobs':[None], 
     'random_state':[0], 
     'verbose':[0], 
     'warm_start':[False], 
     'ccp_alpha':[0.0], 
     'max_samples':[None],   
     'monotonic_cst':[None]
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(et_reg, param_grid=param_grid, cv=tscv5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
# Print the best hyperparameters and the corresponding validation score
print("Best hyperparameters: ", grid_search.best_params_)
#print("Validation score: ", grid_search.best_score_)
# Use the best model to make predictions on the validation set
best_et_reg = grid_search.best_estimator_
y_val_pred = best_et_reg.predict(X_validation)
# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_et = best_et_reg.predict(X_test)

Best hyperparameters:  {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': True, 'random_state': 0, 'verbose': 0, 'warm_start': False}
Validation MSE:  0.1299501105771723
Validation R2:  0.5883830284041117
Validation MAE:  0.3195108842413823


In [162]:
# Create a DataFrame with the feature importance values
feature_importance_et = pd.DataFrame({'Feature': X_train.columns, 'Importance': et.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_et = feature_importance_et.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_et.columns = ['feat', 'imp_et']
# Print the feature importance DataFrame
feature_importance_et.head(30)

,feat,imp_et
0,lag_1,0.212812
1,lag_2,0.043679
2,sugar_tj,0.040398
3,ufv,0.039524
4,flour_lp,0.037218
5,oil2_bol,0.036624
6,oil_cb,0.033458
7,chicken_tj,0.029249
8,chicken_bol,0.027045
9,sugar_lp,0.026732


## **4. Report**

#### End